In [1]:
# Hello there.   This is a do-over of the floating preferred analyzer

# Time to develop clean modules. Let's start with reading in a list of prefs
# The input data consists of three separate tables:
# a) A listing of tickers and their configurations
# b) A list of ratings for each company (equivalently, of the ticker)
# c) The current interest rate environment, which has only the 3-month rate and the prime rate



In [2]:
# Imports go here
import pandas as pd
import requests_cache
# local ones
import prefcode as pc


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Set up cache
session_cached = requests_cache.CachedSession('yfinance.cache',expire_after=3600)
session_uncached = requests_cache.CachedSession('yfinance2.cache',expire_after=30)


In [43]:
# Read the prefs database

print("Setting up databases")
df, interest_db = pc.setup_databases()

print("Fetching price data")
df = pc.fetch_prices(df, session_cached, fetch=True)

Setting up databases
Fetching price data


In [44]:
df.head()

,Ticker,Spread,Type,Mult,Rating,Price
0,ALA.PR.B,266.0,T,NaN,P3L,18.50
1,BAM.PR.B,NaN,P,0.7,P2L,12.66
2,BAM.PR.C,NaN,P,0.7,P2L,12.70
3,BAM.PR.E,NaN,P,1.0,P2L,17.60
4,BAM.PR.K,NaN,P,0.7,P2L,12.70


In [45]:
df.dtypes

Ticker     object
Spread    float64
Type       object
Mult      float64
Rating     object
Price     float64
dtype: object

In [47]:
print("Updating dividends and current yield")
df = pc.update_div_and_yield(df, interest_db,price_column="Price")

Updating dividends and current yield


In [48]:
print("Updating market spread for later calculation")
df2 = pc.update_market_spread(df, interest_db)

print("Dropping prime-related preferreds. Maybe another day")
tdf = df[df['Type'] == 'T'].copy()

print("Calculating scenarios")
scenarios = {"010":  [0.10,  0.05],
             "018":  [0.18,  0.25],
             "030":  [0.30,  0.50],
             "050":  [0.50,  0.20]}

pc.update_expected_yield(tdf, scenarios)

Updating market spread for later calculation
Dropping prime-related preferreds. Maybe another day
Calculating scenarios


In [49]:
print("Here are the highest yield items")
tdf.sort_values(by='ExpYield', ascending=False).head(15)

Here are the highest yield items


,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,010_Yield,018_Yield,030_Yield,050_Yield,ExpYield
21,FN.PR.B,207.0,T,NaN,P3I,13.15,0.5632,4.2829,4.0999,2.5108,4.2284,6.6876,10.5011,6.626660
32,TRP.PR.H,128.0,T,NaN,P2L,12.50,0.3658,2.9264,2.7434,-0.0067,2.8101,6.7576,12.6878,6.618550
29,TA.PR.E,203.0,T,NaN,P3L,13.75,0.5532,4.0233,3.8403,2.3084,3.9706,6.3435,10.0067,6.281160
15,CVE.PR.B,173.0,T,NaN,P2L,14.31,0.4783,3.3424,3.1594,1.4299,3.2654,5.8594,9.8046,5.778465
33,TRP.PR.I,154.0,T,NaN,P2L,14.56,0.4308,2.9588,2.7758,0.8771,2.8741,5.6749,9.8868,5.577190
12,BPO.PR.S,348.0,T,NaN,P3I,19.75,0.9157,4.6365,4.4535,4.1565,4.6238,5.2951,6.3418,5.279685
18,FFH.PR.F,216.0,T,NaN,P3H,16.25,0.5857,3.6043,3.4213,2.3441,3.5682,5.3057,7.9653,5.255165
22,FTS.PR.I,145.0,T,NaN,P3H,15.25,0.4083,2.6774,2.4944,0.6185,2.5925,5.3417,9.4340,5.236700
27,SLF.PR.J,141.0,T,NaN,P2H,15.60,0.3982,2.5526,2.3696,0.5390,2.4928,5.2041,9.2189,5.095980
19,FFH.PR.H,256.0,T,NaN,P3H,17.84,0.6857,3.8436,3.6606,2.9655,3.8198,5.0365,6.9085,5.003175


In [50]:
print("And uniquified by issuer")
# Uniquify by parent
tdf['Parent'] = [x.split('.')[0] for x in tdf['Ticker']]
#tdf
uniq_parent = pc.summarize_best_by_column(tdf,'ExpYield','Parent')

uniq_parent = uniq_parent.reindex(columns=["Ticker", "Rating", 
                                           "Spread","CurYieldPct","ExpYield"])
# make it a bit more readable

uniq_parent['CurYield'] = [round(x,3) for x in uniq_parent['CurYieldPct']]
uniq_parent['ExpYield'] = [round(x,3) for x in uniq_parent['ExpYield']]
uniq_parent.drop(columns=['CurYieldPct'], errors='ignore', inplace=True)
uniq_parent.sort_values(by='ExpYield', ascending=False).head(15)


And uniquified by issuer


,Ticker,Rating,Spread,ExpYield,CurYield
21,FN.PR.B,P3I,207.0,6.627,4.283
32,TRP.PR.H,P2L,128.0,6.619,2.926
29,TA.PR.E,P3L,203.0,6.281,4.023
15,CVE.PR.B,P2L,173.0,5.778,3.342
12,BPO.PR.S,P3I,348.0,5.280,4.636
18,FFH.PR.F,P3H,216.0,5.255,3.604
22,FTS.PR.I,P3H,145.0,5.237,2.677
27,SLF.PR.J,P2H,141.0,5.096,2.553
24,MFC.PR.P,P2I,141.0,4.919,2.489
26,PWF.PR.Q,P2H,160.0,4.887,2.759


In [ ]:
best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield", "MSpread"])

print("And here are the best by rating level")
best_scn_df.sort_values(by=['Rating'])

In [ ]:
# Now, check market spread and flag if below average for the group
# That could result in a re-rating
mdf = pc.calculate_avg_per_rating(tdf, column='MSpread')
mdf